In [ ]:
from google.colab import drive
drive.mount ('/gdrive')

In [ ]:
%cd # gdrive path here

In [ ]:
import pandas as pd
import numpy as np
import stan
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [ ]:
from cmdstanpy import CmdStanModel, set_cmdstan_path, cmdstan_path
import arviz as az
import os

In [ ]:
import cmdstanpy
cmdstanpy.install_cmdstan()

In [ ]:
cmdstan_path()

In [ ]:
nugget = pd.read_csv('nugget_to_python.csv', sep = ";")
data_vec = pd.read_csv('dat_complete_log_to_python.csv', sep = ';') #log PM10 values
ind_miss = pd.read_csv('ind_miss_to_python.csv', sep = ';')
ind_pres = pd.read_csv('ind_pres_to_python.csv', sep = ';')
dati_covariates = pd.read_csv('covariates.csv', sep = ";")
grid = pd.read_csv('grid_UTMcoords.csv', sep = ';')
coord = pd.read_csv('UTMcoords.csv', sep = ';')

In [ ]:
nugget_mat = np.matrix(nugget, dtype=float)
grid_mat = np.matrix(grid, dtype=float)
coord_mat = np.matrix(coord, dtype=float)

data_vettore = np.array(data_vec, dtype=float)
ind_miss = np.array(ind_miss)
ind_pres = np.array(ind_pres)

matrix_dati_covariates=np.matrix(dati_covariates)
quota= np.array (matrix_dati_covariates[0,:], dtype=int)
quota_norm = ((quota-quota.mean())/quota.std())

area_dummies = pd.get_dummies(dati_covariates.iloc[1, :])

zona_dummies = pd.get_dummies(dati_covariates.iloc[2, :])
all_cov = np.matrix(pd.concat([area_dummies.Urbano, area_dummies.Suburbano, 
                               zona_dummies.Industriale, zona_dummies.Traffico],axis=1),dtype=bool)
all_cov = np.concatenate((np.transpose(quota_norm),all_cov),axis=1)

### STAN model

In [ ]:
fourier_model = """

functions {

  vector gp_pred_rng(vector [] x_pred, vector y_obs, vector[] x_obs, real alpha,
                      real rho, real delta, real mu){

      int n_obs = size(x_obs);
      int n_pred = size(x_pred);
      vector[n_pred] y_return;

      {
          matrix[n_obs, n_obs] L;
          vector[n_obs] K_div_y;
          matrix[n_obs,n_pred] k_x;
          matrix[n_obs,n_pred] y_pred;
          vector[n_pred] y_mu;
          matrix[n_pred,n_pred] cov_y;
          matrix[n_pred,n_pred] diag_delta;
          matrix[n_obs, n_obs] K;

          K = gp_exponential_cov(x_obs, alpha, rho);
          L = cholesky_decompose(K);
          K_div_y = mdivide_left_tri_low(L, y_obs);
          K_div_y = mdivide_right_tri_low(K_div_y',L)';
          k_x = gp_exponential_cov(x_obs, x_pred, alpha, rho);

          y_mu = (k_x' * K_div_y) + mu;
          y_pred = mdivide_left_tri_low(L, k_x);
          cov_y = gp_exponential_cov(x_pred, alpha, rho) - y_pred' * y_pred;
          diag_delta = diag_matrix( rep_vector(delta,n_pred));

          y_return = multi_normal_rng(y_mu, cov_y + diag_delta);
      }

      return y_return;

    }
}

data {
  int<lower=0> num_giorni;  //number of obs
  int<lower=0> num_stazioni;  //number of stations
  int<lower=0> d;  //length of vector of basis
  vector[d] vec_k;  //vector of basis
  vector[num_giorni] t;  //time instant
  matrix[num_stazioni,num_stazioni] dist_w;  //distance matrix
  int Ncomp; // Number of non-missing values
  int  Nmiss; // Number of missing values
  int ind_pres[Ncomp, 2];
  int ind_miss[Nmiss, 2];
  vector[Ncomp] dat_complete;
  matrix[num_stazioni, 5] covariates;
  real delta;
  int n_new;
  vector[2] grid[n_new];
  vector[2] coord[num_stazioni];
}


parameters {
  real<lower=0> sigma;
  vector[d] alpha;
  vector[d] beta;
  real<lower=0> a;
  real<lower=0> phi;
  vector[num_stazioni] w;
  vector [Nmiss] dat_miss;
  real beta_0;
  vector[5] beta_cov;
}


transformed parameters {
  real omega = 2*pi()/365;
  vector[num_giorni] fourier;
  matrix[num_giorni,num_stazioni] mu;
  matrix[num_stazioni,num_stazioni] cov_w;
  fourier = rep_vector(0,num_giorni);
  matrix [num_giorni,num_stazioni] y;


  for (i in 1:Ncomp) {
    y[ind_pres[i,1], ind_pres[i,2]] =  dat_complete[i];
  }
  for(i in 1:Nmiss) {
      y[ind_miss[i,1],ind_miss[i,2]] = dat_miss[i];
    }


  for (i in 1:d){
    fourier += alpha[i]*sin(vec_k[i]*omega*t) + beta[i]*cos(vec_k[i]*omega*t);
  }
  

  cov_w = a*exp(-phi*dist_w);
  

  for (j in 1:num_stazioni){
      mu[:,j] = beta_0 + covariates[j,:]*beta_cov + fourier + w[j];
    }
}


model {
  sigma ~ inv_gamma(3,2);

  alpha ~ normal(rep_vector(0,d),1);
  beta ~ normal(rep_vector(0,d),1);

  a ~ inv_gamma(3,2);
  phi ~ beta(7,70);

  beta_0 ~ normal(0, 2);
  beta_cov ~ normal(rep_vector(0,5), 2);

  w ~ multi_normal(rep_vector(0,num_stazioni),cov_w);
  
  for (j in 1:num_stazioni)
    y[:,j] ~ normal(mu[:,j], sqrt(sigma));
}


generated quantities {
  vector[num_giorni*num_stazioni] log_lik;
  {
    matrix [num_giorni, num_stazioni] temp; 
    for (i in 1:365) {
      for (j in 1:num_stazioni) {
      temp[i,j]= normal_lpdf(y[i,j] | mu[i,j], sqrt(sigma));
      }
    }
    log_lik = to_vector(temp);
  }

  vector[n_new] w_pred;
  w_pred = gp_pred_rng(grid, w, coord, sqrt(a), 1/phi, delta, 0);
  
}

"""

stan_file = "./fourier.stan"

with open(stan_file, "w") as f:
    print(fourier_model, file=f)

fourier = CmdStanModel(stan_file=stan_file)

In [ ]:
reg_data = {
    "num_giorni": 365,
    "num_stazioni": 62,
    "d": 3,
    "vec_k": [1,2,4],
    "t": np.arange(1,366),
    "dist_w": nugget_mat,
    "Ncomp": np.shape(ind_pres)[0],
    "Nmiss": np.shape(ind_miss)[0],
    "ind_pres": ind_pres,
    "ind_miss": ind_miss,
    "dat_complete": data_vettore[:,0],
    "covariates": all_cov,
    "delta": 1e-9,
    "n_new": np.shape(grid_mat)[0],
    "grid": grid_mat,
    "coord": coord_mat
}

fit = fourier.sample(data=reg_data, chains=4, parallel_chains=4, 
                iter_warmup=1000, iter_sampling=1000)
fourier_az = az.from_cmdstanpy(fit)

In [ ]:
az.plot_trace(fourier_az, var_names=['beta_cov','beta_0','sigma','a','phi','w','alpha', 'beta','w_pred'], combined=True)

In [ ]:
np.sum(fourier_az.sample_stats.diverging)

### Kriging prediction

In [ ]:
nugget_new = fit.stan_variable(var= "w_pred")

In [ ]:
mean_nugget_new = np.zeros(185)
for i in range(185):
  mean_nugget_new[i] = np.mean(nugget_new[:,i])

In [ ]:
coords_new = pd.read_csv('grid_lat_long_to_python.csv', sep = ';')
coords_new = np.matrix(coords_new, dtype=float)

In [ ]:
plt.scatter(np.array(coords_new[:,0]),np.array(coords_new[:,1]),c=mean_nugget_new)

In [ ]:
pd.DataFrame(mean_nugget_new)